In [1]:
# 금융상품 추천 Random Forest 모델링
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.multioutput import MultiOutputClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from imblearn.over_sampling import SMOTE
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('../2.전처리/data/SCFP/cleaned_scf_data.csv')
df

,연령대분류,교육수준분류,사업농업소득,자본이득소득,연령,총소득,금융위험감수,저축여부,급여소득,금융위험회피,...,가구주성별,결혼상태,자녀수,직업분류1,총자산,LIQ,CDS,NMMF,STOCKS,RETQLIQ
0,5,3,0.0,0.0,70,38804.734469,0,3,0.0,0,...,2,2,2,3,957100.0,1,0,0,0,1
1,5,3,0.0,0.0,70,38264.278557,0,3,0.0,0,...,2,2,2,3,1067300.0,1,0,0,0,1
2,5,3,0.0,0.0,70,36102.454910,0,3,0.0,0,...,2,2,2,3,957200.0,1,0,0,0,1
3,5,3,0.0,0.0,70,33508.266533,0,3,0.0,0,...,2,2,2,3,828600.0,1,0,0,0,1
4,5,3,0.0,0.0,70,35561.998998,0,3,0.0,0,...,2,2,2,3,825600.0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22288,2,2,0.0,0.0,40,38912.825651,0,1,0.0,1,...,2,2,2,3,0.0,0,0,0,0,0
22289,2,2,0.0,0.0,40,33508.266533,0,1,0.0,1,...,2,2,2,3,0.0,0,0,0,0,0
22290,2,2,0.0,0.0,40,38912.825651,0,1,0.0,1,...,2,2,2,3,0.0,0,0,0,0,0
22291,2,2,0.0,0.0,40,38912.825651,0,1,0.0,1,...,2,2,2,3,0.0,0,0,0,0,0


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [4]:
X = df.iloc[:,:-5]
y = df.iloc[:,-5:]
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2)
print(f"훈련 데이터: {train_X.shape[0]}개")
print(f"테스트 데이터: {test_X.shape[0]}개")

훈련 데이터: 17834개
테스트 데이터: 4459개


In [5]:
rf_model = RandomForestClassifier(n_estimators=100,
                                    max_depth=10,          # 깊이 제한
                                    min_samples_split=50,  # 분할 최소 샘플
                                    min_samples_leaf=20,   # 리프 최소 샘플
                                    random_state=42)
rf_model.fit(train_X, train_y)

RandomForestClassifier(max_depth=10, min_samples_leaf=20, min_samples_split=50,
                       random_state=42)

In [6]:
train_pred = rf_model.predict(train_X) #과적합 여부 확인(훈련 성능)
test_pred = rf_model.predict(test_X) #일반화 성능

train_pred_proba = rf_model.predict_proba(train_X)
test_pred_proba = rf_model.predict_proba(train_X)

print(train_pred)

[[1 0 0 0 0]
 [1 0 0 1 1]
 [1 0 0 0 1]
 ...
 [1 0 0 0 1]
 [1 0 0 0 1]
 [1 0 0 0 1]]


In [7]:
print(train_pred_proba)

[array([[9.24410969e-02, 9.07558903e-01],
       [0.00000000e+00, 1.00000000e+00],
       [1.67661528e-03, 9.98323385e-01],
       ...,
       [7.32642125e-03, 9.92673579e-01],
       [6.14753532e-04, 9.99385246e-01],
       [2.39184830e-03, 9.97608152e-01]]), array([[0.9957068 , 0.0042932 ],
       [0.86987746, 0.13012254],
       [0.8061243 , 0.1938757 ],
       ...,
       [0.97866622, 0.02133378],
       [0.82826117, 0.17173883],
       [0.98892967, 0.01107033]]), array([[0.99513347, 0.00486653],
       [0.58008395, 0.41991605],
       [0.86373608, 0.13626392],
       ...,
       [0.95633698, 0.04366302],
       [0.88430386, 0.11569614],
       [0.96649101, 0.03350899]]), array([[0.98441267, 0.01558733],
       [0.49974491, 0.50025509],
       [0.75844885, 0.24155115],
       ...,
       [0.84164397, 0.15835603],
       [0.86158092, 0.13841908],
       [0.90501742, 0.09498258]]), array([[0.84877702, 0.15122298],
       [0.03872468, 0.96127532],
       [0.49882138, 0.50117862],
    

In [8]:
print(test_pred)

[[1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 1]
 ...
 [1 0 0 0 0]
 [1 0 0 1 1]
 [1 0 0 0 1]]


In [9]:
print(test_pred_proba)

[array([[9.24410969e-02, 9.07558903e-01],
       [0.00000000e+00, 1.00000000e+00],
       [1.67661528e-03, 9.98323385e-01],
       ...,
       [7.32642125e-03, 9.92673579e-01],
       [6.14753532e-04, 9.99385246e-01],
       [2.39184830e-03, 9.97608152e-01]]), array([[0.9957068 , 0.0042932 ],
       [0.86987746, 0.13012254],
       [0.8061243 , 0.1938757 ],
       ...,
       [0.97866622, 0.02133378],
       [0.82826117, 0.17173883],
       [0.98892967, 0.01107033]]), array([[0.99513347, 0.00486653],
       [0.58008395, 0.41991605],
       [0.86373608, 0.13626392],
       ...,
       [0.95633698, 0.04366302],
       [0.88430386, 0.11569614],
       [0.96649101, 0.03350899]]), array([[0.98441267, 0.01558733],
       [0.49974491, 0.50025509],
       [0.75844885, 0.24155115],
       ...,
       [0.84164397, 0.15835603],
       [0.86158092, 0.13841908],
       [0.90501742, 0.09498258]]), array([[0.84877702, 0.15122298],
       [0.03872468, 0.96127532],
       [0.49882138, 0.50117862],
    

In [10]:
target_names = {
    'LIQ': '유동성자산',
    'CDS': '양도성예금증서', 
    'NMMF': '비머니마켓펀드',
    'STOCKS': '주식보유',
    'RETQLIQ': '퇴직준비금유동성'
}

print("\n📊 모델 성능 평가")
print("=" * 50)

# 각 타겟별 성능 평가
for i, target in enumerate(y.columns):
    print(f"\n ★{target_names.get(target, target)} ({target})")
    print("-" * 30)
    
    # 정확도
    train_acc = (train_pred[:, i] == train_y.iloc[:, i]).mean()
    test_acc = (test_pred[:, i] == test_y.iloc[:, i]).mean()
    
    print(f"훈련 정확도: {train_acc:.3f}")
    print(f"테스트 정확도: {test_acc:.3f}")

    # 각 타겟마다 별도로 출력 보장
    display(f"=== {target} 완료 ===")  # Jupyter용 강제 출력
    
    # AUC Score
    try:
        train_auc = roc_auc_score(train_y.iloc[:, i], train_pred_proba[i][:, 1])
        test_auc = roc_auc_score(test_y.iloc[:, i], test_pred_proba[i][:, 1])
        print(f"훈련 AUC: {train_auc:.3f}")
        print(f"테스트 AUC: {test_auc:.3f}")
    except:
        print("AUC 계산 불가 (클래스 불균형)")
    
    # Classification Report
    print(f"\n분류 보고서 (테스트 데이터):")
    print(classification_report(test_y.iloc[:, i], test_pred[:, i]))


📊 모델 성능 평가

 ★유동성자산 (LIQ)
------------------------------
훈련 정확도: 0.989
테스트 정확도: 0.989


'=== LIQ 완료 ==='

AUC 계산 불가 (클래스 불균형)

분류 보고서 (테스트 데이터):
              precision    recall  f1-score   support

           0       1.00      0.27      0.42        67
           1       0.99      1.00      0.99      4392

    accuracy                           0.99      4459
   macro avg       0.99      0.63      0.71      4459
weighted avg       0.99      0.99      0.99      4459


 ★양도성예금증서 (CDS)
------------------------------
훈련 정확도: 0.922
테스트 정확도: 0.930


'=== CDS 완료 ==='

AUC 계산 불가 (클래스 불균형)

분류 보고서 (테스트 데이터):
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      4146
           1       0.00      0.00      0.00       313

    accuracy                           0.93      4459
   macro avg       0.46      0.50      0.48      4459
weighted avg       0.86      0.93      0.90      4459


 ★비머니마켓펀드 (NMMF)
------------------------------
훈련 정확도: 0.891
테스트 정확도: 0.885


...

AUC 계산 불가 (클래스 불균형)

분류 보고서 (테스트 데이터):
              precision    recall  f1-score   support

           0       0.90      0.96      0.93      3631
           1       0.76      0.56      0.64       828

    accuracy                           0.88      4459
   macro avg       0.83      0.76      0.79      4459
weighted avg       0.88      0.88      0.88      4459


 ★주식보유 (STOCKS)
------------------------------
훈련 정확도: 0.830
테스트 정확도: 0.827


'=== STOCKS 완료 ==='

AUC 계산 불가 (클래스 불균형)

분류 보고서 (테스트 데이터):
              precision    recall  f1-score   support

           0       0.85      0.92      0.88      3210
           1       0.73      0.60      0.66      1249

    accuracy                           0.83      4459
   macro avg       0.79      0.76      0.77      4459
weighted avg       0.82      0.83      0.82      4459


 ★퇴직준비금유동성 (RETQLIQ)
------------------------------
훈련 정확도: 0.868
테스트 정확도: 0.859


'=== RETQLIQ 완료 ==='

AUC 계산 불가 (클래스 불균형)

분류 보고서 (테스트 데이터):
              precision    recall  f1-score   support

           0       0.88      0.78      0.82      1898
           1       0.85      0.92      0.88      2561

    accuracy                           0.86      4459
   macro avg       0.86      0.85      0.85      4459
weighted avg       0.86      0.86      0.86      4459



In [11]:
# 교차 검증으로 과적합 확인
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(rf_model, X, y, cv=5)
print(f"CV 평균: {cv_scores.mean():.3f}")

CV 평균: 0.510


In [12]:
# SMOTE 적용 테스트
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X, y['STOCKS'])

cv_scores = cross_val_score(
    RandomForestClassifier(n_estimators=50, random_state=42),
    X_balanced, y_balanced, cv=5
)
print(f"SMOTE 적용 CV: {cv_scores.mean():.3f}")

SMOTE 적용 CV: 0.896


# SMOTE적용 후 RandomForest 모델링

In [57]:
import pandas as pd
import numpy as np
import json
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report, hamming_loss, jaccard_score, accuracy_score

In [58]:
# 데이터 로드
df = pd.read_csv('../2.전처리/data/SCFP/cleaned_scf_data.csv')
X = df.iloc[:,:-5]
y = df.iloc[:,-5:]

target_names = {
    'LIQ': '유동성자산',
    'CDS': '양도성예금증서', 
    'NMMF': '비머니마켓펀드',
    'STOCKS': '주식보유',
    'RETQLIQ': '퇴직준비금유동성'
}

PRODUCT_MAPPING = {
    'LIQ': 'bank_deposits',      # 유동성자산 → 예금
    'CDS': 'bank_deposits',      # 양도성예금증서 → 예금
    'NMMF': 'investment_funds',  # 비머니마켓펀드 → 펀드
    'STOCKS': 'investment_funds', # 주식보유 → 펀드
    'RETQLIQ': 'pension_savings' # 퇴직준비금유동성 → 연금저축
}


In [59]:
def load_product_catalog():
    """실제 금융상품 JSON 파일들을 로드"""
    try:
        with open('../1.데이터수집/data/financial_data.json', 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        print("✅ JSON 파일 로드 성공!")
        print(f"📊 데이터 구조: {list(data.keys())}")
        
        # financial_data.json의 실제 구조에 맞게 반환
        return {
            'bank_deposits': data,  # 전체 데이터를 bank_deposits로 사용
        }
    except FileNotFoundError:
        print("⚠️  JSON 파일을 찾을 수 없습니다.")
        return {}
    except Exception as e:
        print(f"⚠️  JSON 로드 오류: {e}")
        return {}


In [60]:
def get_top_products_by_category(catalog, category, top_n=3):
    """카테고리별 상위 상품 추출 (금리 기준)"""
    if category not in catalog or not catalog[category]:
        return []
    
    # JSON 구조 안전하게 접근: products > bank_deposits > result
    try:
        products = catalog[category]['products']['bank_deposits']['result']['baseList']
        options = catalog[category]['products']['bank_deposits']['result']['optionList']
    except KeyError:
        return []
    
    # 상품별 최고 금리 계산
    product_rates = {}
    for option in options:
        product_id = option['fin_prdt_cd']
        rate = option.get('intr_rate2', option.get('intr_rate', 0))
        if product_id not in product_rates or rate > product_rates[product_id]:
            product_rates[product_id] = rate
    
    # 상품 정보와 금리 매칭
    enriched_products = []
    for product in products:
        product_id = product['fin_prdt_cd']
        if product_id in product_rates:
            enriched_products.append({
                'id': product_id,
                'name': product['fin_prdt_nm'],
                'bank': product['kor_co_nm'],
                'rate': product_rates[product_id],
                'join_way': product['join_way'],
                'features': product.get('spcl_cnd', ''),
                'max_limit': product.get('max_limit')
            })
    
    # 금리 순으로 정렬하여 상위 N개 반환
    return sorted(enriched_products, key=lambda x: x['rate'], reverse=True)[:top_n]

In [61]:
# 모델 훈련
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

base_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=50,
    min_samples_leaf=20,
    class_weight='balanced',
    random_state=42
)

multilabel_model = MultiOutputClassifier(base_model)
multilabel_model.fit(train_X, train_y)

MultiOutputClassifier(estimator=RandomForestClassifier(class_weight='balanced',
                                                       max_depth=10,
                                                       min_samples_leaf=20,
                                                       min_samples_split=50,
                                                       random_state=42))

In [62]:
#예측
test_pred = multilabel_model.predict(test_X)

In [63]:
# 성능 평가
exact_match = accuracy_score(test_y, test_pred)

In [64]:
print(f"📊 모델 성능")
print(f"정확한 매칭: {exact_match:.3f}")

📊 모델 성능
정확한 매칭: 0.482


In [65]:
# 실제 상품 추천 함수
def recommend_financial_products(user_features, threshold=0.3):
    """사용자 특성을 바탕으로 실제 금융상품 추천"""
    
    # 상품 카테고리별 추천 확률 예측
    probabilities = multilabel_model.predict_proba(user_features.reshape(1, -1))
    
    # 상품 카탈로그 로드
    catalog = load_product_catalog()
    
    recommendations = []
    
    for i, (label, name) in enumerate(target_names.items()):
        prob = probabilities[i][0][1]  # 보유할 확률
        
        if prob >= threshold:
            # 해당 카테고리의 실제 상품 정보 가져오기
            category = PRODUCT_MAPPING.get(label, 'bank_deposits')
            actual_products = get_top_products_by_category(catalog, category, top_n=3)
            
            recommendations.append({
                'category': name,
                'category_code': label,
                'probability': prob,
                'recommended': True,
                'products': actual_products
            })
        else:
            recommendations.append({
                'category': name,
                'category_code': label,
                'probability': prob,
                'recommended': False,
                'products': []
            })
    
    return sorted(recommendations, key=lambda x: x['probability'], reverse=True)

In [ ]:
# 추천 시스템 데모
print(f"\n🎯 실제 상품 추천 시스템")
print("=" * 50)

sample_user = test_X.iloc[0].values #test데이터의 0번째 사용자
recommendations = recommend_financial_products(sample_user, threshold=0.3)

for rec in recommendations:
    status = "✅ 추천" if rec['recommended'] else "❌ 비추천"
    print(f"\n📂 {rec['category']} ({rec['probability']:.3f}) {status}")
    
    if rec['products']:
        for i, product in enumerate(rec['products'][:2], 1):  # 상위 2개만 표시
            print(f"  {i}. {product['name']}")
            print(f"     {product['bank']} | 금리: {product['rate']:.2f}%")
            print(f"     가입방법: {product['join_way']}")

print(f"\n✨ JSON 연동 금융상품 추천 시스템 완성!")
print("💡 실제 은행 상품 데이터와 연동하여 맞춤형 추천을 제공합니다.")


🎯 실제 상품 추천 시스템
✅ JSON 파일 로드 성공!
📊 데이터 구조: ['last_updated', 'data_source', 'products', 'update_history']

📂 유동성자산 (0.851) ✅ 추천
  1. Sh첫만남우대예금
     수협은행 | 금리: 2.90%
     가입방법: 인터넷,스마트폰
  2. NH고향사랑기부예금
     농협은행주식회사 | 금리: 2.80%
     가입방법: 영업점,인터넷,스마트폰

📂 퇴직준비금유동성 (0.291) ❌ 비추천

📂 주식보유 (0.065) ❌ 비추천

📂 비머니마켓펀드 (0.021) ❌ 비추천

📂 양도성예금증서 (0.004) ❌ 비추천

✨ JSON 연동 금융상품 추천 시스템 완성!
💡 실제 은행 상품 데이터와 연동하여 맞춤형 추천을 제공합니다.


In [67]:
try:
    with open('../1.데이터수집/data/financial_data.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
        
    print("✅ JSON 파일 로드 성공!")
    print(f"📊 데이터 구조: {list(data.keys())}")
        
    # financial_data.json의 실제 구조에 맞게 반환
    print({
            'bank_deposits': data,  # 전체 데이터를 bank_deposits로 사용
        })
except FileNotFoundError:
        print("⚠️  JSON 파일을 찾을 수 없습니다.")
        print({})
except Exception as e:
        print(f"⚠️  JSON 로드 오류: {e}")
        print({})

✅ JSON 파일 로드 성공!
📊 데이터 구조: ['last_updated', 'data_source', 'products', 'update_history']
{'bank_deposits': {'last_updated': '2025-08-06T16:47:09.393530', 'data_source': '금융감독원 금융상품 통합 비교공시', 'products': {'bank_deposits': {'result': {'prdt_div': 'D', 'total_count': 40, 'max_page_no': 1, 'now_page_no': 1, 'err_cd': '000', 'err_msg': '정상', 'baseList': [{'dcls_month': '202507', 'fin_co_no': '0010001', 'fin_prdt_cd': 'WR0001B', 'kor_co_nm': '우리은행', 'fin_prdt_nm': 'WON플러스예금', 'join_way': '인터넷,스마트폰,전화(텔레뱅킹)', 'mtrt_int': '만기 후\n- 1개월이내 : 만기시점약정이율×50%\n- 1개월초과 6개월이내: 만기시점약정이율×30%\n- 6개월초과 : 만기시점약정이율×20%\n\n※ 만기시점 약정이율 : 일반정기예금 금리', 'spcl_cnd': '해당사항 없음', 'join_deny': '1', 'join_member': '실명의 개인', 'etc_note': '- 가입기간: 1~36개월\n- 최소가입금액: 1만원 이상\n- 만기일을 일,월 단위로 자유롭게 선택 가능\n- 만기해지 시 신규일 당시 영업점과 인터넷 홈페이지에 고시된 계약기간별 금리 적용', 'max_limit': None, 'dcls_strt_day': '20250721', 'dcls_end_day': None, 'fin_co_subm_day': '202507210911'}, {'dcls_month': '202507', 'fin_co_no': '0010002', 'fin_prdt_cd': '00320342